In [4]:
#Import the required modules.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm 
import os
import csv

In [5]:
#Fix the randomness.
seed = 1234
torch.manual_seed(1234)


In [6]:
#Download the dataset, and split it into Train, Val, and Test sets.

train_transform = T. Compose ([
# can add additional transforms on images
T.ToTensor () , # convert images to PyTorch tensors
T.Grayscale () , # RGB to grayscale
T.Normalize ( mean =(0.5 ,) , std=(0.5 ,)) # normalization
# speeds up the convergence
# and improves the accuracy
])
val_transform = test_transform = T. Compose ([
T.ToTensor () ,
T.Grayscale () ,
T.Normalize ( mean =(0.5 ,) , std=(0.5 ,))
])

train_set = CIFAR10(root="CIFAR10",train=True,transform = train_transform, download=True) #Get train data set.
train_set_length = int (0.8 * len(train_set)) #Divide train set as train and validation.
val_set_length = len(train_set)-train_set_length

train_set, val_set = random_split(train_set,[train_set_length,val_set_length]) #Randomly split train and validation data sets.
test_set = CIFAR10(root="CIFAR10", train=False,transform=test_transform, download=True) #Get test data set.
#Define the data loaders.
batch_size = 1024
train_loader = DataLoader(train_set, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_set,batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
#Define the ANN
class MyModel(nn.Module):
    def __init__(self,neuron,activation_function,layer) -> None: # Getting inputs.
        super().__init__()
        self.activation_function = activation_function # Giving activation function as parameter.
        self.layer = layer # Giving layer as parameter too.
        if self.layer == 1: # Because of I tried 3 layer, I just build a quick setup with if elses. 
            self.layer1 = nn.Linear(in_features=32*32,out_features=neuron) # If I have 1 layer between input and output, I am directly giving the neuron.
            self.layer2 = nn.Linear(in_features=neuron,out_features=10)
        elif self.layer == 2:
            self.layer1 = nn.Linear(in_features=32*32,out_features=neuron*2) # If I have 2 layer between input and output, I am multplying it with 2 and 1.
            self.layer2 = nn.Linear(in_features=neuron*2,out_features=neuron*1) # For example, if neuron is 100, It becomes 1024->200, 200->100, 100->10
            self.layer3 = nn.Linear(in_features=neuron*1,out_features=10)
        elif self.layer == 3:
            self.layer1 = nn.Linear(in_features=32*32,out_features=neuron*3) # If I have 3 layer between input and output, I am multplying it with 3, 2 ,and 1.
            self.layer2 = nn.Linear(in_features=neuron*3,out_features=neuron*2) # For example, if neuron is 100, It becomes 1024->300, 300->200, 200->100, 100->10.
            self.layer3 = nn.Linear(in_features=neuron*2,out_features=neuron*1)
            self.layer4 = nn.Linear(in_features=neuron*1,out_features=10) # In every case it has to finish 10.
    def forward(self,x):
        x = torch.flatten(x,1)
        if self.layer == 1: # As above, I just build basic if else setup for acivation functions. According to number of layer, it adds activation function between them.
            x = self.layer1(x) # Because of my Python skills not good enough, I prefer building this setup basically and focusing ML part more.
            x = self.activation_function(x)
            x = self.layer2(x)
        elif self.layer == 2:
            x = self.layer1(x)
            x = self.activation_function(x)
            x = self.layer2(x)
            x = self.activation_function(x)
            x = self.layer3(x)
        elif self.layer == 3:
            x = self.layer1(x)
            x = self.activation_function(x)
            x = self.layer2(x)
            x = self.activation_function(x)
            x = self.layer3(x)
            x = self.activation_function(x)
            x = self.layer4(x)
        return x

In [8]:
learning_rates = [1e-1,1e-3,1e-4] 
neurons = [100,150]
activation_functions = [nn.ReLU(),nn.Sigmoid()]
layers = [1,2,3]

In [ ]:
#Instantiate the model and train it for 100 epochs
device = 'cuda' if torch.cuda.is_available() else 'cpu'


num_epochs = 100 # Number of Epoch
modelID = 0 # Initializing model id
results = []
for learning_rate in learning_rates:
    for neuron in neurons:
        for activation_function in activation_functions:
            for layer in layers:
                model = MyModel(neuron=neuron,activation_function=activation_function,layer=layer).to(device)
                best_model = model
                loss_function = nn.CrossEntropyLoss()
                optimizer =  torch.optim.Adam(model.parameters(),lr=learning_rate)
                last_validation_accuracy = 0 # Giving initally low value to replace it quickly.
                last_val_loss = 100 # Giving initally high value to replace it quickly.
                patience = 10 # Defining patience
                print(f'ModelID: {modelID}| Learning_Rate = {learning_rate}\tNeuron = {neuron}\tActivation_function = {activation_function}\tLayer = {layer}')      
                for epoch in tqdm(range(num_epochs)):
                #Training
                    model.train()
                    accum_train_loss = 0
                    for i, (imgs,labels) in enumerate(train_loader,start=1):
                        imgs, labels = imgs.to(device), labels.to(device)
                        output = model(imgs)
                        loss = loss_function(output, labels)

                        #Accumlate the loss
                        accum_train_loss += loss.item()

                        #Backpropagation
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    #Validation
                    model.eval()
                    accum_val_loss = 0
                    correct2 = total2 = 0 # Initializing values to calculate validation accuracy.
                    with torch.no_grad():
                        for j, (imgs,labels) in enumerate(val_loader, start=1):
                            imgs,labels = imgs.to(device),labels.to(device)
                            output = model(imgs)
                            _, predicted_labels = torch.max(output,1)
                            correct2 += (predicted_labels == labels).sum()
                            total2 += labels.size(0)
                            accum_val_loss += loss_function(output,labels).item()
                    validation_accuracy = 100 * correct2/total2 # Calculate validation accuracy like in test accuracy.
                    print(f'Validation Accuracy = {validation_accuracy:.3f}%')
                    #Print statistics of the epoch
                    print(f'Epoch = {epoch} | Train Loss = {accum_train_loss / i:.4f}\tVal Loss = {accum_val_loss / j:.4f}')

                    val_loss = accum_val_loss / j # Getting val_loss for this epoch. 
                    train_loss = accum_train_loss / i # Getting train_loss for this epoch. 

                    if epoch == 0: # For the first epoch, we are definning our first result as our best result.
                        best_validation_accuracy = validation_accuracy
                        best_val_loss = val_loss
                        best_train_loss = train_loss
                        best_values = [learning_rate, neuron, activation_function,layer, epoch, best_validation_accuracy,best_val_loss,best_train_loss,modelID]

                    if validation_accuracy > last_validation_accuracy and val_loss> last_val_loss: 
                        patience -= 1 # If validation accuracy and validation loss higher than previous one loss patience
                    elif validation_accuracy > best_validation_accuracy and val_loss  < best_val_loss:
                        best_model = model # If validation accuracy is better and validation loss is lower, it means it is our new best model.
                        best_validation_accuracy = validation_accuracy
                        best_val_loss = val_loss # Giving values to compare them later.
                        best_train_loss = train_loss # Save the result.
                        best_values = [learning_rate, neuron, activation_function,layer, epoch, best_validation_accuracy,best_val_loss,best_train_loss,modelID]
                    
                    print(f'Patience Left = {patience}')
                    if patience == 0: # If patience is 0, save the best model to a local path.
                        save_path = f"./bestmodels/learning_rate{learning_rate}/neuron{neuron}/activation_function{activation_function}/layer{layer}"
                        os.makedirs(save_path, exist_ok=True)
                        torch.save(best_model.state_dict(),f"{save_path}/model.pth")
                        results.append(best_values) # Append our best values to a list
                        break
                    if num_epochs - 1 == epoch: # If number of epoch finishes, save the best model to a local path. 
                        save_path = f"./bestmodels/learning_rate{learning_rate}/neuron{neuron}/activation_function{activation_function}/layer{layer}"
                        os.makedirs(save_path, exist_ok=True)
                        torch.save(best_model.state_dict(),f"{save_path}/model.pth")
                        results.append(best_values)  # Append our best values to a list
                    last_val_loss = val_loss # Giving values to another variable to compare them in next run. 
                    last_validation_accuracy = validation_accuracy
                print(best_values)
                modelID += 1 # Increase model id.
file = open('train_results.csv', 'w+', newline ='') # Print all results to a excel file.
with file:     
    write = csv.writer(file) 
    write.writerows(results) 

In [ ]:
#Compute the test accuracy 
test_results = []
modelID = 0 # Initializing model id
for learning_rate in learning_rates:
    for neuron in neurons:
        for activation_function in activation_functions:
            for layer in layers:
                from_path = f"./bestmodels/learning_rate{learning_rate}/neuron{neuron}/activation_function{activation_function}/layer{layer}/model.pth"
                print(f'Learning_Rate = {learning_rate}\tNeuron = {neuron}\tActivation_function = {activation_function}\tLayer = {layer}')
                print(f"./bestmodels/learning_rate{learning_rate}/neuron{neuron}/activation_function{activation_function}/layer{layer}/model.pth")
                model = MyModel(neuron=neuron,activation_function=activation_function,layer=layer).to(device) # Creating a model

                model.load_state_dict(torch.load(from_path)) # Loading our best models from the local path.
                model.eval()

                with torch.no_grad():
                    correct = total = 0
                    for images, labels in test_loader:
                        images, labels = images.to(device),labels.to(device)
                        output = model(images)
                        _, predicted_labels = torch.max(output,1)
                        correct += (predicted_labels == labels).sum()
                        total += labels.size(0)
                print(f'Test Accuracy = {100 * correct/total:.3f}%')
                test_result = [modelID,100 * correct/total]
                test_results.append(test_result) # Getting test results in the list.
                modelID += 1

file = open('test_results.csv', 'w+', newline ='') 
with file:     # Printing results to an excel file.
    write = csv.writer(file) 
    write.writerows(test_results) 